In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
import logging

In [3]:

# Set up logging to silence verbose TensorFlow Datasets output
logging.basicConfig(level=logging.INFO)
tf.get_logger().setLevel(logging.ERROR)

In [4]:
NUM_CLASSES = 120
IMG_SIZE = 224
BATCH_SIZE = 16 # FIXED: Reduced from 32 to 16 to lower immediate memory use during training.
EPOCHS = 3

In [5]:
def load_stanford_dogs():
    """Loads the dataset and returns splits and metadata."""
    print("Loading Stanford Dogs dataset...")
    (ds_train, ds_test), info = tfds.load(
        "stanford_dogs",
        split=['train', 'test'],
        as_supervised=True,  # (image, label) format
        with_info=True,
        shuffle_files=True
    )
    print(f"Dataset loaded. Total classes: {info.features['label'].num_classes}")
    return ds_train, ds_test


In [6]:
def preprocess_image(image, label):
    """Resize and normalize image for MobileNetV2 [ -1, 1 ]."""
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32)
    # Apply MobileNetV2-specific normalization
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, label


In [7]:
def prepare_dataset(ds, shuffle=False):
    """Configures dataset for performance with map, shuffle, batch, and prefetch."""
    ds = ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    # FIXED: The ds.cache() call was removed from here.
    # This ensures the dataset streams data from the source files instead of
    # trying to load the entire processed dataset into RAM, resolving the crash.

    if shuffle:
        ds = ds.shuffle(buffer_size=1000)

    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds


In [8]:
def build_dog_classifier_model():
    """Uses MobileNetV2 for transfer learning by freezing the base and adding a new head."""
    print("Building MobileNetV2 model for transfer learning...")

    # Load MobileNetV2 base model pre-trained on ImageNet
    base_model = MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False, # Exclude the default classification head
        weights='imagenet'
    )

    # Freeze the convolutional base
    base_model.trainable = False

    # Create the new classification layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x) # Reduce feature maps
    predictions = Dense(NUM_CLASSES, activation='softmax')(x) # Final classification layer

    model = Model(inputs=base_model.input, outputs=predictions)
    return model


In [9]:
def train_and_evaluate(model, train_ds, test_ds):
    """Compiles, fits, and evaluates the model."""
    print("Compiling model...")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    print(f"Starting training for {EPOCHS} epochs...")
    model.fit(
        train_ds,
        epochs=EPOCHS,
        validation_data=test_ds
    )

    print("\nEvaluating model on test set...")
    loss, accuracy = model.evaluate(test_ds)
    print(f"Test Accuracy: {accuracy*100:.2f}%")


In [10]:
if __name__ == '__main__':
    try:
        # Load and prepare raw data splits
        ds_train_raw, ds_test_raw = load_stanford_dogs()

        # Configure datasets for efficient deep learning
        train_ds = prepare_dataset(ds_train_raw, shuffle=True)
        test_ds = prepare_dataset(ds_test_raw)

        # Build the transfer learning model
        model = build_dog_classifier_model()

        # Train and evaluate
        train_and_evaluate(model, train_ds, test_ds)

    except Exception as e:
        print(f"\nAn error occurred: {e}")
        print("Ensure you have TensorFlow and TensorFlow Datasets installed.")


Loading Stanford Dogs dataset...
Dataset loaded. Total classes: 120
Building MobileNetV2 model for transfer learning...
Compiling model...
Starting training for 3 epochs...
Epoch 1/3
750/750 ━━━━━━━━━━━━━━━━━━━━ 69s 67ms/step - accuracy: 0.5713 - loss: 1.8611 - val_accuracy: 0.8079 - val_loss: 0.6109
Epoch 2/3
750/750 ━━━━━━━━━━━━━━━━━━━━ 38s 48ms/step - accuracy: 0.8905 - loss: 0.3535 - val_accuracy: 0.8190 - val_loss: 0.5730
Epoch 3/3
750/750 ━━━━━━━━━━━━━━━━━━━━ 41s 49ms/step - accuracy: 0.9373 - loss: 0.2142 - val_accuracy: 0.8197 - val_loss: 0.5758

Evaluating model on test set...
537/537 ━━━━━━━━━━━━━━━━━━━━ 14s 27ms/step - accuracy: 0.8259 - loss: 0.5665
Test Accuracy: 81.97%
